In [ ]:
import React, { useState, useEffect } from "react";
import { Card, CardHeader, CardTitle, CardContent } from "@/components/ui/card";
import { ChevronDown, ChevronUp, Globe, AlertCircle, Download } from "lucide-react";

// Declare the global property to fix TypeScript errors
declare global {
  interface Window {
    _schoolGradeMap: Record<string, any>;
  }
}

const AlertsDashboard = () => {
  const [district, setDistrict] = useState("");
  const [school, setSchool] = useState("");
  const [grade, setGrade] = useState("");

  const [districtOptions, setDistrictOptions] = useState([]);
  const [schoolOptions, setSchoolOptions] = useState([]);
  const [gradeOptions, setGradeOptions] = useState([]);

  const [analysisData, setAnalysisData] = useState(null);
  const [loading, setLoading] = useState(true); // Start with loading true for initial data fetch
  const [error, setError] = useState(null);
  const [isGlobalView, setIsGlobalView] = useState(false);
  const [allSchoolOptions, setAllSchoolOptions] = useState([]);
  const [allGradeOptions, setAllGradeOptions] = useState([]);
  
  // Store hierarchical data from the backend
  const [hierarchicalData, setHierarchicalData] = useState([]);

  const [showFilters, setShowFilters] = useState(true);
  const [isInitialLoad, setIsInitialLoad] = useState(true);
  const [loadTimer, setLoadTimer] = useState(null);

  // Combined fetch function to reduce multiple requests
  const fetchInitialData = async () => {
    setLoading(true);
    setError(null);

    // Clear any existing timer
    if (loadTimer) {
      clearTimeout(loadTimer);
    }

    try {
      // Try to fetch filter options first
      const filterRes = await fetch(`http://127.0.0.1:8001/api/filter-options`);
      
      if (filterRes.ok) {
        const filterData = await filterRes.json();
        
        // Log the structure for debugging
        console.log("API RESPONSE STRUCTURE:", filterData);
        
        setDistrictOptions(filterData.districts || []);
        setAllSchoolOptions(filterData.schools || []);
        setAllGradeOptions(filterData.grades || []); // Store all grades globally
        setGradeOptions(filterData.grades || []);    // Initialize with all grades
        setSchoolOptions([]);
        
        // Store school to grade mapping for direct lookup
        const schoolGradeMap = new Map();
        
        try {
          // Try to extract school-to-grade mappings from the raw data
          filterData.schools.forEach(school => {
            const schoolName = school.value;
            // Create a direct query to the backend for each school's grades
            // We'll do this when a school is selected, not now
          });
        } catch (err) {
          console.error("Error processing school grades:", err);
        }
        
        // If filter options were successful, try to fetch global analysis
        try {
          const analysisRes = await fetch(`http://127.0.0.1:8001/api/global-analysis`);
          if (analysisRes.ok) {
            const analysisData = await analysisRes.json();
            setAnalysisData(analysisData);
            setIsGlobalView(true);
            setIsInitialLoad(false);
          } else {
            throw new Error("Failed to load dashboard data");
          }
        } catch (analysisErr) {
          console.error("Error fetching analysis:", analysisErr);
        }
      } else {
        // If server is not ready, set a simple loading message
        setError("Server is starting up. Please wait...");
        
        // Schedule a single retry with increasing timeout
        const timer = setTimeout(() => {
          fetchInitialData();
        }, 3000);
        
        setLoadTimer(timer);
      }
    } catch (err) {
      console.error("Error fetching initial data:", err);
      
      // Schedule a single retry
      const timer = setTimeout(() => {
        fetchInitialData();
      }, 3000);
      
      setLoadTimer(timer);
    } finally {
      setLoading(false);
    }
  };

  useEffect(() => {
    fetchInitialData();
    
    // Cleanup function to clear any pending timeouts
    return () => {
      if (loadTimer) {
        clearTimeout(loadTimer);
      }
    };
  }, []);

  useEffect(() => {
    const trimmedDistrict = district.trim();
    if (trimmedDistrict) {
      const filteredSchools = allSchoolOptions.filter(
        (s) => s.district.trim() === trimmedDistrict
      );
      setSchoolOptions(filteredSchools);
    } else {
      setSchoolOptions(allSchoolOptions); // Show all schools if no district selected
    }
  
    // Don't reset selected school unless it becomes invalid
    if (
      school &&
      trimmedDistrict &&
      !allSchoolOptions.find(
        (s) => s.district.trim() === trimmedDistrict && s.value === school
      )
    ) {
      setSchool("");
    }
  
    // Always reset grade when district changes
    setGrade("");
    
    // Reset grade options to all available when district changes
    setGradeOptions(allGradeOptions);
  }, [district, allSchoolOptions]);

  // Update grade options when school changes - SIMPLIFIED APPROACH
  useEffect(() => {
    // Skip if no school selected
    if (!school) {
      setGradeOptions(allGradeOptions);
      setGrade("");
      return;
    }

    console.log("School selected:", school);
    
    // DIRECT DATABASE QUERY APPROACH
    const getGradesForSchool = async () => {
      // Array to hold valid grades for this school
      const validGrades = [];
      
      // Common grade levels to check - we'll only check these common ones to avoid excessive 500 errors
      const commonGrades = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'];
      
      // Set initial state to loading
      setGradeOptions([]);
      
      let hasFoundAnyGrades = false;
      
      // Check each common grade to see if it exists for this school
      for (const grade of commonGrades) {
        try {
          // Make a request to check if this grade exists for this school
          const response = await fetch(`http://127.0.0.1:8001/api/analysis?school_name=${encodeURIComponent(school)}&grade_level=${grade}`);
          
          // Only process successful responses
          if (response.ok) {
            const data = await response.json();
            
            // If we get data back with students, this grade exists for this school
            if (data && data.summary_statistics && data.summary_statistics.total_students > 0) {
              console.log(`Grade ${grade} exists for this school`);
              
              // Find the matching grade object from allGradeOptions
              const gradeObj = allGradeOptions.find(g => g.value === grade);
              if (gradeObj) {
                validGrades.push(gradeObj);
                hasFoundAnyGrades = true;
              }
            }
          }
          // Silently ignore 500 errors - they just mean this grade doesn't exist for this school
        } catch (error) {
          // Silently ignore errors
        }
      }
      
      // Once we've checked all common grades, update the dropdown
      if (validGrades.length > 0) {
        console.log(`Found ${validGrades.length} valid grades for this school:`, validGrades.map(g => g.value));
        setGradeOptions(validGrades);
      } else {
        // If we didn't find any valid grades, show all available grades as a fallback
        console.log("No specific grades found for this school, using all grades as fallback");
        setGradeOptions(allGradeOptions);
      }
      
      // Always reset the selected grade when school changes
      setGrade("");
    };
    
    // Execute the function
    getGradesForSchool();
  }, [school, allGradeOptions]);

  const fetchAnalysis = async () => {
    setLoading(true);
    setError(null);
    setIsGlobalView(false);
  
    try {
      const params = new URLSearchParams();
      if (district) params.append("district_name", district.trimEnd() + " ");
      if (school) params.append("school_name", school);
      if (grade) params.append("grade_level", grade);
  
      const res = await fetch(`http://127.0.0.1:8001/api/analysis?${params.toString()}`, {
        method: "GET",
        headers: { Accept: "application/json" },
      });
  
      if (!res.ok) {
        if (res.status === 503) {
          setError("Server is starting up. Please wait...");
          return;
        } else {
          throw new Error("Failed to load dashboard data");
        }
      }

      const contentType = res.headers.get("content-type");
      if (!contentType || !contentType.includes("application/json")) {
        throw new Error("Invalid response format");
      }
  
      const data = await res.json();
      setAnalysisData(data);
      setError(null);
    } catch (err) {
      console.error("Error fetching analysis:", err);
      if (!err.message.includes("starting up")) {
        setError("Failed to load data. Please try again.");
      }
    } finally {
      setLoading(false);
    }
  };
  
  const fetchGlobalAnalysis = async () => {
    setLoading(true);
    setError(null);
    setIsGlobalView(true);
    
    try {
      const res = await fetch(`http://127.0.0.1:8001/api/global-analysis`, {
        method: "GET",
        headers: { Accept: "application/json" },
      });

      if (!res.ok) {
        if (res.status === 503) {
          setError("Server is starting up. Please wait...");
          return;
        } else {
          throw new Error("Failed to load dashboard data");
        }
      }

      const contentType = res.headers.get("content-type");
      if (!contentType || !contentType.includes("application/json")) {
        throw new Error("Invalid response format");
      }

      const data = await res.json();
      setAnalysisData(data);
      setError(null);
    } catch (err) {
      console.error("Error fetching global analysis:", err);
      if (!err.message.includes("starting up")) {
        setError("Failed to load data. Please try again.");
      }
    } finally {
      setLoading(false);
    }
  };
  
  const resetFilters = () => {
    setDistrict("");
    setSchool("");
    setGrade("");
    setGradeOptions(allGradeOptions); // Reset to all available grades
    setIsGlobalView(false);
    fetchGlobalAnalysis();
  };

  const handleDownloadReport = async (reportType = "summary") => {
    try {
      const queryParams = new URLSearchParams();
      if (grade) queryParams.append("grade_level", grade);
      if (school) queryParams.append("school_name", school);
      if (district) queryParams.append("district_name", district.trimEnd() + " ");
  
      const res = await fetch(`http://127.0.0.1:8001/api/download/${reportType}?${queryParams}`, {
        method: "GET",
      });
  
      if (!res.ok) throw new Error(`Failed to download ${reportType} report`);
  
      const blob = await res.blob();
      const url = window.URL.createObjectURL(blob);
  
      const a = document.createElement("a");
      a.href = url;
      a.download = `${reportType}_report.xlsx`;
      document.body.appendChild(a);
      a.click();
      a.remove();
      window.URL.revokeObjectURL(url);
    } catch (err) {
      console.error("Download error:", err);
      alert(`Failed to download ${reportType} report: ${err.message}`);
    }
  };
  
  const handleDownloadBelow85Report = () => {
    handleDownloadReport("below_85");
  };

  // Render loading skeleton for cards
  const renderSkeletonCards = () => {
    return (
      <>
        {[1, 2, 3, 4, 5].map((i) => (
          <Card key={i} className="animate-pulse">
            <CardHeader><div className="h-5 bg-gray-200 rounded w-24"></div></CardHeader>
            <CardContent><div className="h-8 bg-gray-200 rounded w-16"></div></CardContent>
          </Card>
        ))}
      </>
    );
  };

  // Render school options with unique keys
  const renderSchoolOptions = () => {
    return (
      <>
        <option value="">Select School</option>
        {schoolOptions.map((s, index) => (
          <option 
            key={`${s.value}-${s.district}-${index}`} 
            value={s.value}
          >
            {s.label}
          </option>
        ))}
      </>
    );
  };

  return (
    <div className="min-h-screen bg-gray-50/50">
      <div className="container mx-auto px-4 py-8 max-w-full">
        <div className="mb-6">
          <div className="flex justify-between items-center">
            <div>
              <h1 className="text-3xl font-bold">Alerts Dashboard</h1>
              <p className="text-muted-foreground">Monitor alerts and notifications</p>
            </div>
            <div className="flex items-center gap-2">
              <button
                onClick={() => setShowFilters(!showFilters)}
                className="flex items-center gap-1 text-sm text-blue-600 hover:underline ml-4"
              >
                {showFilters ? "Hide Filters" : "Show Filters"}
                {showFilters ? <ChevronUp size={18} /> : <ChevronDown size={18} />}
              </button>
            </div>
          </div>
          <div className="w-full h-0.5 bg-gray-300 mt-1"></div>
        </div>

        <div className="flex w-full min-h-[calc(100vh-6rem)]">
          {showFilters && (
            <div className="w-64 p-4 bg-white shadow rounded-md mr-4 h-[calc(100vh-6rem)] overflow-y-auto">
              <div className="mb-4">
                <label className="block font-semibold mb-1">District</label>
                <select
                  value={district}
                  onChange={(e) => setDistrict(e.target.value)}
                  className="w-full p-2 border rounded"
                  disabled={isInitialLoad}
                >
                  <option value="">Select District</option>
                  {districtOptions.map((d) => (
                    <option key={d.value} value={d.value}>
                      {d.label}
                    </option>
                  ))}
                </select>
              </div>
              
              <div className="mb-4">
                <label className="block font-semibold mb-1">School</label>
                <select
                  value={school}
                  onChange={(e) => setSchool(e.target.value)}
                  className="w-full p-2 border rounded"
                  disabled={!schoolOptions.length || isInitialLoad}
                >
                  {renderSchoolOptions()}
                </select>
              </div>
              <div className="mb-4">
                <label className="block font-semibold mb-1">Grade</label>
                <select
                  value={grade}
                  onChange={(e) => setGrade(e.target.value)}
                  className="w-full p-2 border rounded"
                  disabled={!gradeOptions.length || isInitialLoad}
                >
                  <option value="">Select Grade</option>
                  {gradeOptions.map((g) => (
                    <option key={g.value} value={g.value}>
                      {g.label}
                    </option>
                  ))}
                </select>
              </div>
              <div className="flex gap-2">
                <button
                  onClick={fetchAnalysis}
                  className={`bg-blue-600 text-white px-4 py-2 rounded hover:bg-blue-700 w-full ${
                    isInitialLoad ? "opacity-50 cursor-not-allowed" : ""
                  }`}
                  disabled={isInitialLoad}
                >
                  Search
                </button>
                <button
                  onClick={resetFilters}
                  className={`bg-gray-300 text-gray-800 px-4 py-2 rounded hover:bg-gray-400 w-full ${
                    isInitialLoad ? "opacity-50 cursor-not-allowed" : ""
                  }`}
                  disabled={isInitialLoad}
                >
                  Reset
                </button>
              </div>
            </div>
          )}

          {/* Main Dashboard */}
          <div className="flex-1 grid grid-cols-1 sm:grid-cols-2 md:grid-cols-3 lg:grid-cols-4 xl:grid-cols-5 gap-4 p-4">
            {/* Server starting notification */}
            {error && (
              <div className="col-span-full mb-4">
                <div className="bg-yellow-50 border-l-4 border-yellow-500 p-4">
                  <div className="flex">
                    <div className="flex-shrink-0">
                      <svg className="animate-spin h-5 w-5 text-yellow-500" xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24">
                        <circle className="opacity-25" cx="12" cy="12" r="10" stroke="currentColor" strokeWidth="4"></circle>
                        <path className="opacity-75" fill="currentColor" d="M4 12a8 8 0 018-8V0C5.373 0 0 5.373 0 12h4zm2 5.291A7.962 7.962 0 014 12H0c0 3.042 1.135 5.824 3 7.938l3-2.647z"></path>
                      </svg>
                    </div>
                    <div className="ml-3">
                      <p className="text-sm text-yellow-700">{error}</p>
                    </div>
                  </div>
                </div>
              </div>
            )}

            {isGlobalView && analysisData && (
              <div className="col-span-full mb-4">
                <div className="bg-blue-50 border-l-4 border-blue-500 p-4">
                  <div className="flex">
                    <div className="flex-shrink-0">
                      <Globe className="h-5 w-5 text-blue-500" />
                    </div>
                    <div className="ml-3">
                      <p className="text-sm text-blue-700">
                        Viewing Global Analysis - Showing data for all districts, schools, and grades
                      </p>
                    </div>
                  </div>
                </div>
              </div>
            )}
            
            {/* Loading state */}
            {(loading || isInitialLoad) && !error && (
              <>
                <div className="flex justify-center items-center col-span-full mb-4">
                  <div className="flex items-center justify-center gap-2">
                    <svg className="animate-spin h-8 w-8 text-blue-500" xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24">
                      <circle className="opacity-25" cx="12" cy="12" r="10" stroke="currentColor" strokeWidth="4"></circle>
                      <path className="opacity-75" fill="currentColor" d="M4 12a8 8 0 018-8V0C5.373 0 0 5.373 0 12h4zm2 5.291A7.962 7.962 0 014 12H0c0 3.042 1.135 5.824 3 7.938l3-2.647z"></path>
                    </svg>
                    <p className="text-blue-700">Loading dashboard data...</p>
                  </div>
                </div>
                {renderSkeletonCards()}
              </>
            )}
          
            {/* Dashboard cards - only show when we have data */}
            {!loading && !isInitialLoad && analysisData && (
              <>
                <Card className="bg-orange-50 border border-orange-200">
                  <CardHeader className="pb-2">
                    <CardTitle className="flex items-center gap-2">
                      Total Students
                    </CardTitle>
                  </CardHeader>
                  <CardContent className="flex justify-between items-center">
                    <span className="text-xl font-semibold">
                      {analysisData.summary_statistics.total_students}
                    </span>
                    <button 
                      onClick={() => handleDownloadReport("summary")}
                      className="text-xs bg-orange-500 text-white p-1 rounded flex items-center gap-1"
                      title="Download Summary Report"
                    >
                      <Download size={12} />
                      Export
                    </button>
                  </CardContent>
                </Card>
                <Card>
                  <CardHeader><CardTitle>At Risk Students</CardTitle></CardHeader>
                  <CardContent>{analysisData.summary_statistics.at_risk_students}</CardContent>
                </Card>
                <Card className="bg-orange-50 border-orange-200">
                  <CardHeader className="pb-2">
                    <CardTitle className="flex items-center gap-2">
                      <AlertCircle size={16} className="text-orange-500" />
                      Below 85% Attendance
                    </CardTitle>
                  </CardHeader>
                  <CardContent className="flex justify-between items-center">
                    <span className="text-xl font-semibold">{analysisData.summary_statistics.below_85_students}</span>
                    <button
                      onClick={handleDownloadBelow85Report}
                      className="text-xs bg-orange-500 text-white p-1 rounded flex items-center gap-1"
                      title="Download Below 85% Report"
                    >
                      <Download size={12} />
                      Export
                    </button>
                  </CardContent>
                </Card>
                <Card>
                  <CardHeader><CardTitle>Critical Risk Students</CardTitle></CardHeader>
                  <CardContent>{analysisData.summary_statistics.critical_risk_students}</CardContent>
                </Card>
                <Card>
                  <CardHeader><CardTitle>Tier 4 Students</CardTitle></CardHeader>
                  <CardContent>{analysisData.summary_statistics.tier4_students}</CardContent>
                </Card>

                {/* Improved UI for Key Insights and Recommendations */}
                <div className="col-span-full mt-6">
                  <div className="grid grid-cols-1 lg:grid-cols-2 gap-6">
                    {/* Key Insights */}
                    {analysisData.key_insights?.length > 0 && (
                      <Card className="bg-gradient-to-br from-blue-50 to-indigo-50 shadow-md border border-blue-100 overflow-hidden">
                        <CardHeader className="bg-blue-100/60 pb-3">
                          <CardTitle className="text-blue-800 flex items-center gap-2">
                            <svg xmlns="http://www.w3.org/2000/svg" className="h-5 w-5" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                              <path strokeLinecap="round" strokeLinejoin="round" strokeWidth={2} d="M13 16h-1v-4h-1m1-4h.01M21 12a9 9 0 11-18 0 9 9 0 0118 0z" />
                            </svg>
                            AI DRIVEN Key Insights
                          </CardTitle>
                        </CardHeader>
                        <CardContent className="pt-4">
                          <ul className="space-y-2">
                            {analysisData.key_insights.map((item, index) => (
                              <li key={index} className="flex items-start gap-2">
                                <span className="text-blue-500 mt-1">•</span>
                                <span dangerouslySetInnerHTML={{
                                  __html: item.insight.replace(/(\d+(\.\d+)?%?)/g, "<strong class='text-blue-700'>$1</strong>")
                                }} />
                              </li>
                            ))}
                          </ul>
                        </CardContent>
                      </Card>
                    )}
                    
                    {/* AI Recommendations */}
                    {analysisData.recommendations?.length > 0 && (
                      <Card className="bg-gradient-to-br from-green-50 to-teal-50 shadow-md border border-green-100 overflow-hidden">
                        <CardHeader className="bg-green-100/60 pb-3">
                          <CardTitle className="text-green-800 flex items-center gap-2">
                            <svg xmlns="http://www.w3.org/2000/svg" className="h-5 w-5" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                              <path strokeLinecap="round" strokeLinejoin="round" strokeWidth={2} d="M9 12l2 2 4-4m6 2a9 9 0 11-18 0 9 9 0 0118 0z" />
                            </svg>
                            AI Recommendations
                          </CardTitle>
                        </CardHeader>
                        <CardContent className="pt-4">
                          <ul className="space-y-2">
                            {analysisData.recommendations.map((item, index) => (
                              <li key={index} className="flex items-start gap-2">
                                <span className="text-green-500 mt-1">•</span>
                                <span dangerouslySetInnerHTML={{
                                  __html: item.recommendation.replace(/(\d+(\.\d+)?%?)/g, "<strong class='text-green-700'>$1</strong>")
                                }} />
                              </li>
                            ))}
                          </ul>
                        </CardContent>
                      </Card>
                    )}
                  </div>
                </div>
              </>
            )}
          </div>
        </div>
      </div>
    </div>
  );
};

export default AlertsDashboard;